In [28]:
## Importing relevant libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from sklearn.metrics import accuracy_score, f1_score

In [2]:
## Connecting drive to colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
## Loading the dataset and embeddings

embeddings = np.load(r"/content/drive/MyDrive/Colab Notebooks/_Haider/data/embeddings.npy")
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/_Haider/data/articles_gender.csv")
print(f"Shape of embeddings: {embeddings.shape}")
print(f"Shape of df: {df.shape}")

Shape of embeddings: (142426, 300)
Shape of df: (142426, 8)


In [4]:
## Declaring tergets and label encoding them

y_sentiment = df['sentiment'].values
sentiment_label_encoder = LabelEncoder()
y_sentiment = sentiment_label_encoder.fit_transform(y_sentiment)
y_sentiment = to_categorical(y_sentiment, num_classes=3)

y_gender = df['gender'].values
gender_label_encoder = LabelEncoder()
y_gender = gender_label_encoder.fit_transform(y_gender)
y_gender = to_categorical(y_gender, num_classes=3)

In [5]:
## Splitting the data into training and testing sets

X_train, X_test, y_sentiment_train, y_sentiment_test, y_gender_train, y_gender_test = train_test_split(embeddings, y_sentiment, y_gender,
                                                                                                       test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_sentiment_train shape:", y_sentiment_train.shape)
print("y_sentiment_test shape:", y_sentiment_test.shape)
print("y_gender_train shape:", y_gender_train.shape)
print("y_gender_test shape:", y_gender_test.shape)

X_train shape: (113940, 300)
X_test shape: (28486, 300)
y_sentiment_train shape: (113940, 3)
y_sentiment_test shape: (28486, 3)
y_gender_train shape: (113940, 3)
y_gender_test shape: (28486, 3)


In [6]:
## Reshaping the embeddings to 3D

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (113940, 1, 300)
X_test shape: (28486, 1, 300)


In [7]:
## Creating an empty DataFrame to store the results

results_df = pd.DataFrame(columns=['Model', 'Accuracy (Sentiment)', 'F1 Score (Sentiment)', 'Accuracy (Gender)', 'F1 Score (Gender)'])
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)


In [8]:
## Defining the Bidirectional RNN model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(Bidirectional(SimpleRNN(128), input_shape=(X_train.shape[1], X_train.shape[2])))
sentiment_model.add(tf.keras.layers.Dense(64, activation='relu'))
sentiment_model.add(tf.keras.layers.Dense(3, activation='softmax'))  # For multi-class sentiment classification

## Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the sentiment prediction model
sentiment_model.fit(X_train, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = sentiment_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 32s 7ms/step - loss: 0.5778 - accuracy: 0.7381 - val_loss: 0.5687 - val_accuracy: 0.7412
Epoch 2/100
3027/3027 [==============================] - 19s 6ms/step - loss: 0.5625 - accuracy: 0.7477 - val_loss: 0.5597 - val_accuracy: 0.7462
Epoch 3/100
3027/3027 [==============================] - 19s 6ms/step - loss: 0.5549 - accuracy: 0.7504 - val_loss: 0.5555 - val_accuracy: 0.7492
Epoch 4/100
3027/3027 [==============================] - 20s 7ms/step - loss: 0.5488 - accuracy: 0.7541 - val_loss: 0.5588 - val_accuracy: 0.7487
Epoch 5/100
3027/3027 [==============================] - 19s 6ms/step - loss: 0.5430 - accuracy: 0.7574 - val_loss: 0.5566 - val_accuracy: 0.7467
Epoch 6/100
891/891 [==============================] - 2s 2ms/step


In [9]:
## Defining the Bidirectional RNN model for gender prediction
gender_model = Sequential()
gender_model.add(Bidirectional(SimpleRNN(128), input_shape=(X_train.shape[1], X_train.shape[2])))
gender_model.add(tf.keras.layers.Dense(64, activation='relu'))
gender_model.add(tf.keras.layers.Dense(3, activation='softmax'))  # For multi-class gender classification

## Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the gender prediction model
gender_model.fit(X_train, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = gender_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 23s 6ms/step - loss: 0.5594 - accuracy: 0.8014 - val_loss: 0.5510 - val_accuracy: 0.8028
Epoch 2/100
3027/3027 [==============================] - 20s 7ms/step - loss: 0.5424 - accuracy: 0.8075 - val_loss: 0.5469 - val_accuracy: 0.8056
Epoch 3/100
3027/3027 [==============================] - 20s 6ms/step - loss: 0.5353 - accuracy: 0.8095 - val_loss: 0.5433 - val_accuracy: 0.8050
Epoch 4/100
3027/3027 [==============================] - 21s 7ms/step - loss: 0.5296 - accuracy: 0.8111 - val_loss: 0.5448 - val_accuracy: 0.8063
Epoch 5/100
3027/3027 [==============================] - 20s 7ms/step - loss: 0.5236 - accuracy: 0.8136 - val_loss: 0.5452 - val_accuracy: 0.8069
Epoch 6/100
891/891 [==============================] - 2s 2ms/step


In [11]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['Bidirectional SimpleRNN', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679


In [12]:
## Defining the deeper Bidirectional RNN model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
sentiment_model.add(Dropout(0.2))
sentiment_model.add(Bidirectional(LSTM(64)))
sentiment_model.add(Dropout(0.2))
sentiment_model.add(tf.keras.layers.Dense(32, activation='relu'))
sentiment_model.add(tf.keras.layers.Dense(3, activation='softmax'))  # For multi-class sentiment classification

## Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the sentiment prediction model
sentiment_model.fit(X_train, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = sentiment_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 42s 11ms/step - loss: 0.5820 - accuracy: 0.7349 - val_loss: 0.5669 - val_accuracy: 0.7458
Epoch 2/100
3027/3027 [==============================] - 31s 10ms/step - loss: 0.5653 - accuracy: 0.7440 - val_loss: 0.5597 - val_accuracy: 0.7489
Epoch 3/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5593 - accuracy: 0.7467 - val_loss: 0.5572 - val_accuracy: 0.7488
Epoch 4/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5536 - accuracy: 0.7506 - val_loss: 0.5511 - val_accuracy: 0.7516
Epoch 5/100
3027/3027 [==============================] - 29s 10ms/step - loss: 0.5458 - accuracy: 0.7545 - val_loss: 0.5518 - val_accuracy: 0.7506
Epoch 6/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5383 - accuracy: 0.7576 - val_loss: 0.5545 - val_accuracy: 0.7479
Epoch 7/100
3027/3027 [==============================] - 29s 10ms/step - loss: 0.5296 - accuracy: 0.7624 - val_loss: 0.55

In [13]:
## Defining the deeper Bidirectional RNN model for gender prediction
gender_model = Sequential()
gender_model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
gender_model.add(Dropout(0.2))
gender_model.add(Bidirectional(LSTM(64)))
gender_model.add(Dropout(0.2))
gender_model.add(tf.keras.layers.Dense(32, activation='relu'))
gender_model.add(tf.keras.layers.Dense(3, activation='softmax'))  # For multi-class gender classification

## Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the gender prediction model
gender_model.fit(X_train, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = gender_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 38s 10ms/step - loss: 0.5644 - accuracy: 0.7999 - val_loss: 0.5512 - val_accuracy: 0.8052
Epoch 2/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5453 - accuracy: 0.8067 - val_loss: 0.5441 - val_accuracy: 0.8059
Epoch 3/100
3027/3027 [==============================] - 29s 10ms/step - loss: 0.5377 - accuracy: 0.8094 - val_loss: 0.5447 - val_accuracy: 0.8059
Epoch 4/100
3027/3027 [==============================] - 29s 10ms/step - loss: 0.5312 - accuracy: 0.8110 - val_loss: 0.5467 - val_accuracy: 0.8070
Epoch 5/100
3027/3027 [==============================] - 29s 10ms/step - loss: 0.5245 - accuracy: 0.8131 - val_loss: 0.5382 - val_accuracy: 0.8079
Epoch 6/100
3027/3027 [==============================] - 30s 10ms/step - loss: 0.5171 - accuracy: 0.8156 - val_loss: 0.5409 - val_accuracy: 0.8079
Epoch 7/100
3027/3027 [==============================] - 29s 10ms/step - loss: 0.5101 - accuracy: 0.8172 - val_loss: 0.

In [14]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['Bidirectional LSTM', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679
1,Bidirectional LSTM,0.755283,0.746789,0.812399,0.774131


In [15]:
## Defining the deeper Bidirectional GRU model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
sentiment_model.add(Dropout(0.2))
sentiment_model.add(Bidirectional(GRU(64)))
sentiment_model.add(Dropout(0.2))
sentiment_model.add(tf.keras.layers.Dense(32, activation='relu'))
sentiment_model.add(tf.keras.layers.Dense(3, activation='softmax'))

## Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the sentiment prediction model
sentiment_model.fit(X_train, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = sentiment_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 41s 11ms/step - loss: 0.5821 - accuracy: 0.7351 - val_loss: 0.5608 - val_accuracy: 0.7467
Epoch 2/100
3027/3027 [==============================] - 27s 9ms/step - loss: 0.5671 - accuracy: 0.7427 - val_loss: 0.5581 - val_accuracy: 0.7500
Epoch 3/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5624 - accuracy: 0.7452 - val_loss: 0.5557 - val_accuracy: 0.7501
Epoch 4/100
3027/3027 [==============================] - 29s 9ms/step - loss: 0.5564 - accuracy: 0.7489 - val_loss: 0.5556 - val_accuracy: 0.7510
Epoch 5/100
3027/3027 [==============================] - 30s 10ms/step - loss: 0.5513 - accuracy: 0.7506 - val_loss: 0.5543 - val_accuracy: 0.7521
Epoch 6/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5457 - accuracy: 0.7550 - val_loss: 0.5512 - val_accuracy: 0.7541
Epoch 7/100
3027/3027 [==============================] - 27s 9ms/step - loss: 0.5400 - accuracy: 0.7572 - val_loss: 0.5537

In [16]:
## Defining the deeper Bidirectional GRU model for gender prediction
gender_model = Sequential()
gender_model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
gender_model.add(Dropout(0.2))
gender_model.add(Bidirectional(GRU(64)))
gender_model.add(Dropout(0.2))
gender_model.add(tf.keras.layers.Dense(32, activation='relu'))
gender_model.add(tf.keras.layers.Dense(3, activation='softmax'))

## Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the gender prediction model
gender_model.fit(X_train, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = gender_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 37s 10ms/step - loss: 0.5628 - accuracy: 0.7997 - val_loss: 0.5488 - val_accuracy: 0.8059
Epoch 2/100
3027/3027 [==============================] - 27s 9ms/step - loss: 0.5460 - accuracy: 0.8072 - val_loss: 0.5476 - val_accuracy: 0.8072
Epoch 3/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5399 - accuracy: 0.8091 - val_loss: 0.5435 - val_accuracy: 0.8059
Epoch 4/100
3027/3027 [==============================] - 27s 9ms/step - loss: 0.5347 - accuracy: 0.8099 - val_loss: 0.5402 - val_accuracy: 0.8077
Epoch 5/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5280 - accuracy: 0.8128 - val_loss: 0.5416 - val_accuracy: 0.8081
Epoch 6/100
3027/3027 [==============================] - 27s 9ms/step - loss: 0.5231 - accuracy: 0.8134 - val_loss: 0.5371 - val_accuracy: 0.8075
Epoch 7/100
3027/3027 [==============================] - 28s 9ms/step - loss: 0.5173 - accuracy: 0.8154 - val_loss: 0.5391 

In [17]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['Bidirectional GRU', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679
1,Bidirectional LSTM,0.755283,0.746789,0.812399,0.774131
2,Bidirectional GRU,0.754722,0.745121,0.812855,0.776389


In [22]:
# Defining the LSTM model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
sentiment_model.add(Dense(64, activation='relu'))
sentiment_model.add(Dense(3, activation='softmax'))  # For multi-class sentiment classification

# Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the sentiment prediction model
sentiment_model.fit(X_train, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

# Making predictions on the test data
y_pred = sentiment_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 24s 6ms/step - loss: 0.5791 - accuracy: 0.7358 - val_loss: 0.5615 - val_accuracy: 0.7481
Epoch 2/100
3027/3027 [==============================] - 19s 6ms/step - loss: 0.5640 - accuracy: 0.7446 - val_loss: 0.5579 - val_accuracy: 0.7499
Epoch 3/100
3027/3027 [==============================] - 17s 6ms/step - loss: 0.5568 - accuracy: 0.7486 - val_loss: 0.5618 - val_accuracy: 0.7435
Epoch 4/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5497 - accuracy: 0.7526 - val_loss: 0.5517 - val_accuracy: 0.7519
Epoch 5/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5421 - accuracy: 0.7563 - val_loss: 0.5490 - val_accuracy: 0.7551
Epoch 6/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5333 - accuracy: 0.7602 - val_loss: 0.5574 - val_accuracy: 0.7496
Epoch 7/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5244 - accuracy: 0.7644 - val_loss: 0.5563 -

In [23]:
# Defining the LSTM model for gender prediction
gender_model = Sequential()
gender_model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
gender_model.add(Dense(64, activation='relu'))
gender_model.add(Dense(3, activation='softmax'))  # For multi-class gender classification

# Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the gender prediction model
gender_model.fit(X_train, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

# Making predictions on the test data
y_pred = gender_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 20s 5ms/step - loss: 0.5618 - accuracy: 0.8004 - val_loss: 0.5486 - val_accuracy: 0.8046
Epoch 2/100
3027/3027 [==============================] - 17s 6ms/step - loss: 0.5431 - accuracy: 0.8072 - val_loss: 0.5448 - val_accuracy: 0.8056
Epoch 3/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5353 - accuracy: 0.8098 - val_loss: 0.5421 - val_accuracy: 0.8061
Epoch 4/100
3027/3027 [==============================] - 17s 6ms/step - loss: 0.5280 - accuracy: 0.8126 - val_loss: 0.5388 - val_accuracy: 0.8077
Epoch 5/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5202 - accuracy: 0.8150 - val_loss: 0.5380 - val_accuracy: 0.8088
Epoch 6/100
3027/3027 [==============================] - 17s 6ms/step - loss: 0.5121 - accuracy: 0.8178 - val_loss: 0.5389 - val_accuracy: 0.8091
Epoch 7/100
3027/3027 [==============================] - 18s 6ms/step - loss: 0.5026 - accuracy: 0.8206 - val_loss: 0.5411 -

In [24]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['LSTM', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679
1,Bidirectional LSTM,0.755283,0.746789,0.812399,0.774131
2,Bidirectional GRU,0.754722,0.745121,0.812855,0.776389
3,LSTM,0.753282,0.745075,0.813066,0.775306


In [25]:
# Defining the GRU model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2])))
sentiment_model.add(Dense(64, activation='relu'))
sentiment_model.add(Dense(3, activation='softmax'))  # For multi-class sentiment classification

# Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the sentiment prediction model
sentiment_model.fit(X_train, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

# Making predictions on the test data
y_pred = sentiment_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 22s 6ms/step - loss: 0.5784 - accuracy: 0.7375 - val_loss: 0.5721 - val_accuracy: 0.7387
Epoch 2/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5635 - accuracy: 0.7450 - val_loss: 0.5577 - val_accuracy: 0.7483
Epoch 3/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5568 - accuracy: 0.7494 - val_loss: 0.5613 - val_accuracy: 0.7496
Epoch 4/100
3027/3027 [==============================] - 14s 5ms/step - loss: 0.5503 - accuracy: 0.7519 - val_loss: 0.5574 - val_accuracy: 0.7516
Epoch 5/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5439 - accuracy: 0.7557 - val_loss: 0.5583 - val_accuracy: 0.7516
Epoch 6/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5367 - accuracy: 0.7595 - val_loss: 0.5497 - val_accuracy: 0.7529
Epoch 7/100
3027/3027 [==============================] - 17s 6ms/step - loss: 0.5290 - accuracy: 0.7637 - val_loss: 0.5537 -

In [26]:
# Defining the GRU model for gender prediction
gender_model = Sequential()
gender_model.add(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2])))
gender_model.add(Dense(64, activation='relu'))
gender_model.add(Dense(3, activation='softmax'))  # For multi-class gender classification

# Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the gender prediction model
gender_model.fit(X_train, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

# Making predictions on the test data
y_pred = gender_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 20s 5ms/step - loss: 0.5591 - accuracy: 0.8002 - val_loss: 0.5529 - val_accuracy: 0.8036
Epoch 2/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5421 - accuracy: 0.8074 - val_loss: 0.5455 - val_accuracy: 0.8053
Epoch 3/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5342 - accuracy: 0.8101 - val_loss: 0.5414 - val_accuracy: 0.8070
Epoch 4/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5277 - accuracy: 0.8123 - val_loss: 0.5427 - val_accuracy: 0.8073
Epoch 5/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5211 - accuracy: 0.8143 - val_loss: 0.5402 - val_accuracy: 0.8055
Epoch 6/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5145 - accuracy: 0.8161 - val_loss: 0.5388 - val_accuracy: 0.8069
Epoch 7/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5068 - accuracy: 0.8179 - val_loss: 0.5395 -

In [27]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['GRU', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679
1,Bidirectional LSTM,0.755283,0.746789,0.812399,0.774131
2,Bidirectional GRU,0.754722,0.745121,0.812855,0.776389
3,LSTM,0.753282,0.745075,0.813066,0.775306
4,GRU,0.754581,0.746650,0.809977,0.768481


In [36]:
# Reshape the input data to have the correct shape
X_train_reshaped = X_train.reshape(X_train.shape[0], 300, 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], 300, 1)

# Rest of the code remains the same

# Defining the 1D CNN model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=(300, 1)))
sentiment_model.add(MaxPooling1D(pool_size=2))
sentiment_model.add(Conv1D(64, kernel_size=3, activation='relu'))
sentiment_model.add(GlobalMaxPooling1D())
sentiment_model.add(Dense(3, activation='softmax'))  # For multi-class sentiment classification

# Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the sentiment prediction model
sentiment_model.fit(X_train_reshaped, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

# Making predictions on the test data
y_pred = sentiment_model.predict(X_test_reshaped)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 22s 5ms/step - loss: 0.7101 - accuracy: 0.6169 - val_loss: 0.6870 - val_accuracy: 0.6431
Epoch 2/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6890 - accuracy: 0.6461 - val_loss: 0.6738 - val_accuracy: 0.6618
Epoch 3/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6813 - accuracy: 0.6560 - val_loss: 0.6704 - val_accuracy: 0.6654
Epoch 4/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6756 - accuracy: 0.6603 - val_loss: 0.6679 - val_accuracy: 0.6658
Epoch 5/100
3027/3027 [==============================] - 17s 5ms/step - loss: 0.6717 - accuracy: 0.6634 - val_loss: 0.6850 - val_accuracy: 0.6392
Epoch 6/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.6676 - accuracy: 0.6673 - val_loss: 0.6601 - val_accuracy: 0.6732
Epoch 7/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6647 - accuracy: 0.6696 - val_loss: 0.6624 -

In [37]:
# Reshape the input data to have the correct shape
X_train_reshaped = X_train.reshape(X_train.shape[0], 300, 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], 300, 1)

# Rest of the code remains the same

# Defining the 1D CNN model for gender prediction
gender_model = Sequential()
gender_model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=(300, 1)))
gender_model.add(MaxPooling1D(pool_size=2))
gender_model.add(Conv1D(64, kernel_size=3, activation='relu'))
gender_model.add(GlobalMaxPooling1D())
gender_model.add(Dense(3, activation='softmax'))  # For multi-class gender classification

# Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the gender prediction model
gender_model.fit(X_train_reshaped, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

# Making predictions on the test data
y_pred = gender_model.predict(X_test_reshaped)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 18s 6ms/step - loss: 0.6556 - accuracy: 0.7711 - val_loss: 0.6506 - val_accuracy: 0.7719
Epoch 2/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6491 - accuracy: 0.7711 - val_loss: 0.6426 - val_accuracy: 0.7719
Epoch 3/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6447 - accuracy: 0.7710 - val_loss: 0.6429 - val_accuracy: 0.7720
Epoch 4/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.6419 - accuracy: 0.7711 - val_loss: 0.6381 - val_accuracy: 0.7720
Epoch 5/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6384 - accuracy: 0.7711 - val_loss: 0.6351 - val_accuracy: 0.7722
Epoch 6/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6359 - accuracy: 0.7714 - val_loss: 0.6356 - val_accuracy: 0.7722
Epoch 7/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.6327 - accuracy: 0.7721 - val_loss: 0.6304 -

In [38]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['1D CNN', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679
1,Bidirectional LSTM,0.755283,0.746789,0.812399,0.774131
2,Bidirectional GRU,0.754722,0.745121,0.812855,0.776389
3,LSTM,0.753282,0.745075,0.813066,0.775306
4,GRU,0.754581,0.746650,0.809977,0.768481
5,1D CNN,0.676894,0.647825,0.771502,0.677732


In [39]:
## Defining the SimpleRNN model for sentiment prediction
sentiment_model = Sequential()
sentiment_model.add(SimpleRNN(128, input_shape=(X_train.shape[1], X_train.shape[2])))
sentiment_model.add(tf.keras.layers.Dense(64, activation='relu'))
sentiment_model.add(tf.keras.layers.Dense(3, activation='softmax'))  # For multi-class sentiment classification

## Compiling the sentiment prediction model and setting up early stopping
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the sentiment prediction model
sentiment_model.fit(X_train, y_sentiment_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = sentiment_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_sentiment = accuracy_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels)
weighted_f1_score_sentiment = f1_score(np.argmax(y_sentiment_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 19s 6ms/step - loss: 0.5770 - accuracy: 0.7376 - val_loss: 0.5626 - val_accuracy: 0.7458
Epoch 2/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5618 - accuracy: 0.7478 - val_loss: 0.5612 - val_accuracy: 0.7517
Epoch 3/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5535 - accuracy: 0.7508 - val_loss: 0.5603 - val_accuracy: 0.7487
Epoch 4/100
3027/3027 [==============================] - 15s 5ms/step - loss: 0.5474 - accuracy: 0.7547 - val_loss: 0.5631 - val_accuracy: 0.7438
Epoch 5/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5416 - accuracy: 0.7574 - val_loss: 0.5559 - val_accuracy: 0.7486
Epoch 6/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5368 - accuracy: 0.7611 - val_loss: 0.5570 - val_accuracy: 0.7487
Epoch 7/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5314 - accuracy: 0.7628 - val_loss: 0.5589 -

In [40]:
## Defining the SimpleRNN model for gender prediction
gender_model = Sequential()
gender_model.add(SimpleRNN(128, input_shape=(X_train.shape[1], X_train.shape[2])))
gender_model.add(tf.keras.layers.Dense(64, activation='relu'))
gender_model.add(tf.keras.layers.Dense(3, activation='softmax'))  # For multi-class gender classification

## Compiling the gender prediction model and setting up early stopping
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## Training the gender prediction model
gender_model.fit(X_train, y_gender_train, epochs=100, batch_size=32, validation_split=0.15, callbacks=[early_stopping])

## Making predictions on the test data
y_pred = gender_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

## Calculate the test accuracy and the weighted F1 score
test_accuracy_gender = accuracy_score(np.argmax(y_gender_test, axis=1), y_pred_labels)
weighted_f1_score_gender = f1_score(np.argmax(y_gender_test, axis=1), y_pred_labels, average='weighted')

Epoch 1/100
3027/3027 [==============================] - 18s 5ms/step - loss: 0.5580 - accuracy: 0.8013 - val_loss: 0.5519 - val_accuracy: 0.8036
Epoch 2/100
3027/3027 [==============================] - 17s 6ms/step - loss: 0.5420 - accuracy: 0.8069 - val_loss: 0.5627 - val_accuracy: 0.8002
Epoch 3/100
3027/3027 [==============================] - 20s 7ms/step - loss: 0.5342 - accuracy: 0.8107 - val_loss: 0.5439 - val_accuracy: 0.8073
Epoch 4/100
3027/3027 [==============================] - 17s 5ms/step - loss: 0.5272 - accuracy: 0.8126 - val_loss: 0.5412 - val_accuracy: 0.8081
Epoch 5/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5211 - accuracy: 0.8146 - val_loss: 0.5457 - val_accuracy: 0.8060
Epoch 6/100
3027/3027 [==============================] - 16s 5ms/step - loss: 0.5154 - accuracy: 0.8158 - val_loss: 0.5449 - val_accuracy: 0.8084
Epoch 7/100
891/891 [==============================] - 2s 2ms/step


In [41]:
## Adding the performance in the results dataframe

results_df.loc[len(results_df)] = ['SimpleRNN', test_accuracy_sentiment, weighted_f1_score_sentiment, test_accuracy_gender, weighted_f1_score_gender]
results_df

,Model,Accuracy (Sentiment),F1 Score (Sentiment),Accuracy (Gender),F1 Score (Gender)
0,Bidirectional SimpleRNN,0.752615,0.743027,0.806045,0.757679
1,Bidirectional LSTM,0.755283,0.746789,0.812399,0.774131
2,Bidirectional GRU,0.754722,0.745121,0.812855,0.776389
3,LSTM,0.753282,0.745075,0.813066,0.775306
4,GRU,0.754581,0.746650,0.809977,0.768481
5,1D CNN,0.676894,0.647825,0.771502,0.677732
6,SimpleRNN,0.752966,0.746102,0.812259,0.775542


In [43]:
## saving results
results_df.to_csv(r"/content/drive/MyDrive/Colab Notebooks/_Haider/data/results.csv", index=False)